In [ ]:
from set_home_directory import get_project_root_homedir_in_sys_path
project_root, main_dir = get_project_root_homedir_in_sys_path("inter_areal_predictability")
if project_root is None:
    raise RuntimeError(f"Project root not found: ensure a folder named '{project_root}' exists in one of the sys.path entries.")
print("Project root found:", project_root)


import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import seaborn as sns
import pickle
import sys
import os 

os.chdir(main_dir)
fig_dir = os.path.join(project_root, 'results/paper_figures/')

import sys
sys.path.insert(0,os.path.join(main_dir,'utils/'))
sys.path.insert(0,main_dir)
from utils.fig_3_functions import residualize_within_condition_multi
import utils.mouse_data_functions as cs
from utils.fig_4_functions import make_mouse_df_stimulus_comparisons, fig4_violinplot,add_stars_2_sets,add_violin_custom_colors,plot_neuronal_properties_comparison,make_monkey_df_stimulus_comparisons
from utils.fig_4_functions import add_violin_custom_colors_3, add_group_pairwise_stars_2_sets
from utils.fig_3_functions import color_label


In [ ]:
save_figs = False

mouse_stats_path = os.path.join(project_root, 'results/fig_4',f'mouse_stats.pkl')
with open(mouse_stats_path, 'rb') as f:
	mouse_stats = pickle.load(f)


monkey_names = ['L','A']
all_monkey_stats = {}
for monkey_name in monkey_names:
    monkey_stats_path = os.path.join(project_root, 'results/fig_4',f'monkey_{monkey_name}_stats.pkl')
    with open(monkey_stats_path, 'rb') as f:
        all_monkey_stats[monkey_name] = pickle.load(f)

## Mouse dataset type comparisons

### plotting

In [ ]:
df_mouse_all = make_mouse_df_stimulus_comparisons(mouse_stats)
dfmouse = df_mouse_all[df_mouse_all.control_shuffle==False].reset_index()

variable1='Direction'
variable2='Dataset Type'
filt_df_mouse = dfmouse[(dfmouse.SNR>2)&(dfmouse['Split-half r']>0.8)]
filt_df_mouse = filt_df_mouse[filt_df_mouse['Mouse_Name'].isin(['MP031','MP032','MP033'])].reset_index()
filt_df_mouse_null = df_mouse_all[(df_mouse_all.SNR>2)&(df_mouse_all['Split-half r']>0.8)&(df_mouse_all.control_shuffle==True)]
filt_df_mouse_null = filt_df_mouse_null[filt_df_mouse_null['Mouse_Name'].isin(['MP031','MP032','MP033'])].reset_index()
dataset_types = ['ori32', 'natimg32']
directions= ['L4→L2/3', 'L2/3→L4']
palette=['#72BEB7','#EDAEAE']

In [ ]:
#Fig 4B
neuron_property='EV'
fig, ax = plt.subplots(figsize=((2.85,1.8)))
fontsize=7
fig4_violinplot(filt_df_mouse, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax,
                label_order=directions, hue_order=dataset_types,
                gap=-0.1,width=0.65,cut=0,
                linewidth=0, saturation=1, palette=['#72BEB7','#EDAEAE']
                )

ax.legend_.remove()
color_label(ax, palette,fontsize,y_offset=-0.11, predictor_color='black')

ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
add_violin_custom_colors(ax)
sns.despine()
ax.spines[:].set_linewidth(0.3)
ax.set_ylabel('EV Fraction', fontsize=fontsize, labelpad=0)

add_stars_2_sets(filt_df_mouse, neuron_property, x=variable1, x_order=directions,
                 hue=variable2, ax=ax, height1=0.85, height2=0.85, perm_type='ind', hierarchical=True)

data = filt_df_mouse_null['EV']
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.1, color='blue', label='shuffle\ncontrol IQR',
        linewidth=0,
           )
if save_figs is True:
        plt.savefig(fig_dir +'fig4_mouse_dataset_comparisons_EV_control.pdf',transparent=True,bbox_inches='tight')

plt.show()

### supplemental

In [ ]:
#Fig Supp 5B
variable1='Area'
variable2='Dataset Type'
dataset_types = ['ori32', 'natimg32']
directions= ['L2/3','L4']
figsize=(2,1.3)

mouse_args = {'height1':1.05, 'height2':1.05}

neuron_property='Split-half r'
fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_mouse, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=[0.8,0.9,1], ylabel='split-half r', animal='mouse', 
                                        stats_args=mouse_args, central_tendency='mean')
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)

if save_figs is True:
    plt.savefig(fig_dir +'fig4_supp_mouse_dataset_comparisons_reli.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig Supp 5C
neuron_property='SNR'
fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_mouse, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types,
                                        ylabel=None, log_scale=True, stats_args={'height1':0.8, 'height2':0.8})

ax.set_ylim(top=100)
if save_figs is True:
    plt.savefig(fig_dir +'fig4_supp_mouse_dataset_comparisons_SNR.pdf',transparent=True,bbox_inches='tight')

plt.show()

In [ ]:
#Fig Supp 5D
neuron_property='max corr. val'
fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_mouse, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=None, ylabel='max corr. val.', animal='mouse',stats_args={'height1':0.9, 'height2':0.9})
if save_figs is True:
    plt.savefig(fig_dir +'fig4_supp_mouse_dataset_comparisons_max_corr_val.pdf',transparent=True,bbox_inches='tight')

plt.show()

## Monkey L dataset type comparisons

### plotting

In [ ]:
monkey_stats = all_monkey_stats['L']
df_monkey_all = make_monkey_df_stimulus_comparisons(monkey_stats)
filt_df_monkey = df_monkey_all[(df_monkey_all.control_shuffle==False)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
filt_df_monkey_null= df_monkey_all[(df_monkey_all.control_shuffle==True)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
dataset_types=['RF_thin', 'RF_large', 'SNR']
variable1='Direction'
variable2='Dataset Type'
directions = ['V1→V4','V4→V1']
figsize=(3,2)


In [ ]:
#Fig 4D
neuron_property='EV'
fig, ax = plt.subplots(figsize=((2.85,1.8)))
fontsize=7
filt_df_monkey_median = filt_df_monkey.groupby(['Direction', 'Area','Dataset Type', 'Date','Neuron distribution ID','control_shuffle'])['EV'].median().reset_index()

fig4_violinplot(filt_df_monkey_median, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, label_order=directions, 
                hue_order=dataset_types, gap=-0.1,width=0.7,cut=0,
                linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
                )
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV Fraction', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax)
add_group_pairwise_stars_2_sets(filt_df_monkey, neuron_property, x=variable1, x_order=directions,
                       hue=variable2, hue_order=dataset_types, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
                       stars1_positions=[0.16,0.23,0.30],
                       stars2_positions=[0.69,0.77,0.83])
data = filt_df_monkey_null['EV']
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.1, color='blue', label='shuffle\ncontrol IQR',
        linewidth=0,
           )
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)
if save_figs is True:
    plt.savefig(fig_dir +'fig4_monkey_dataset_comparisons_EV_control.pdf',transparent=True,bbox_inches='tight')
plt.show()



### supplemental

In [ ]:
variable1='Area'
variable2='Dataset Type'
dataset_types = ['RF_thin', 'RF_large', 'SNR']
directions= ['V4','V1']
figsize=(1.7,1.3)
fontsize=7

In [ ]:
#Fig Supp 5H
stats_args = {'fontsize': fontsize*0.8, 'height1':1.02, 'height2': 1.12, 'height3': 1.02, 'stars1_positions': [0.18,0.26,0.34],'stars2_positions':[0.68,0.75,0.83],'bar_height':0.02 }
neuron_property='Split-half r'

fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_monkey, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=[0.8,0.9,1], ylabel='split-half r', animal='monkey', stats_args=stats_args)
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
        ax.set_ylim(top=1)
ax.plot([-0.2,0.2], # -0.095, units are width of the axes
        [1.101, 1.101], # 0 to 500, units are same as the y-axis
        clip_on=False,
        transform=ax.get_xaxis_transform(),
        color='black', linewidth=0.7)
ax.plot([0.804,1.2], # -0.095, units are width of the axes
        [1.101, 1.101], # 0 to 500, units are same as the y-axis
        clip_on=False,
        transform=ax.get_xaxis_transform(),
        color='black', linewidth=0.7)
if save_figs is True:
        plt.savefig(fig_dir +'fig4_supp_monkey_dataset_comparisons_reli.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig Supp 5I
stats_args = {'fontsize': fontsize*0.8, 'height1':0.92, 'height2': 1.02, 'height3': 0.92, 
              'stars1_positions': [0.18,0.26,0.34],'stars2_positions':[0.68,0.75,0.83],'bar_height':0.02 }
neuron_property='SNR'
fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_monkey, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=None, ylabel=None, animal='monkey', stats_args=stats_args,
                                        log_scale=True)
# ax.set_ylim(top=100)
if save_figs is True:
    plt.savefig(fig_dir +'fig4_supp_monkey_dataset_comparisons_SNR.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig Supp 5J
stats_args = {'fontsize': fontsize*0.8, 'height1':0.92, 'height2': 1.02, 'height3': 0.92,'stars1_positions': [0.18,0.26,0.34],'stars2_positions':[0.68,0.75,0.83],'bar_height':0.02 }
neuron_property='max corr. val'

fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_monkey, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=None, ylabel='max corr. val.', animal='monkey', stats_args=stats_args)
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)
if save_figs is True:
    plt.savefig(fig_dir +'fig4_supp_monkey_dataset_comparisons_max_corr_vals.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
## residuals 
monkey_stats = all_monkey_stats['L']
covs = ['SNR', 'Split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials']

df_monkey_all = make_monkey_df_stimulus_comparisons(monkey_stats, get_variances=True)
filt_df_monkey = df_monkey_all[(df_monkey_all.control_shuffle==False)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
filt_df_monkey_null= df_monkey_all[(df_monkey_all.control_shuffle==True)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)

dataset_types=['RF_thin', 'RF_large', 'SNR']
variable1='Direction'
variable2='Dataset Type'
directions = ['V1→V4','V4→V1']
figsize=(3,2)



In [ ]:
#Fig Supp 5K
res_raw  = residualize_within_condition_multi(filt_df_monkey, y='EV', xs=covs, cond='Direction',
                                              flag_col='control_shuffle', flag_keep=False,
                                              standardize=True, model='ols')

# model on log1p for the 3 skewed covs (same function, via callable or precomputed columns)
def xs_logs(g):
    return pd.DataFrame({
        'logSNR': np.log1p(g['SNR']),
        'split' : g['Split-half r'],
        'logVarTime': np.log1p(g['Variance w/in trial\nacross timepoints']),
        'logVarTrials': np.log1p(g['Variance w/in timepoint\nacross trials']),
    }, index=g.index)

res_log = residualize_within_condition_multi(filt_df_monkey, y='EV', xs=xs_logs, cond='Direction',
                                             flag_col='control_shuffle', flag_keep=False,
                                             standardize=True, model='ols')

# compare residuals on keep rows
keep = (filt_df_monkey['control_shuffle']==False)

neuron_property = 'EV_resid'
figsize=(1.7,1.3)

fig, ax = plt.subplots(figsize=figsize)
fontsize=7
fig4_violinplot(res_raw, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, label_order=directions, 
                hue_order=dataset_types, gap=-0.1,width=0.7,cut=0,
                linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
                )
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV residuals', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax)
add_group_pairwise_stars_2_sets(res_raw, neuron_property, x=variable1, x_order=directions,
                       hue=variable2, hue_order=dataset_types, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
                       stars1_positions=[0.16,0.23,0.30],
                       stars2_positions=[0.69,0.77,0.83])
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)
ax.set_ylim(top=0.65)
if save_figs:
    plt.savefig(os.path.join(fig_dir, f'R2.6_macaque_L_ev_resid_covs_direction.pdf'), dpi=500, bbox_inches='tight', transparent=True)

## Monkey A dataset type comparisons

### plotting

In [ ]:
#monkey A only 
monkey_name = 'A'
monkey_stats = all_monkey_stats[monkey_name]
df_monkey_all = make_monkey_df_stimulus_comparisons(monkey_stats)
filt_df_monkey = df_monkey_all[(df_monkey_all.control_shuffle==False)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
filt_df_monkey_null= df_monkey_all[(df_monkey_all.control_shuffle==True)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
dataset_types=['RF_thin', 'RF_large', 'SNR']
variable1='Direction'
variable2='Dataset Type'
directions = ['V1→V4','V4→V1']
figsize=(3,2)


In [ ]:
#Fig Supp 5F
neuron_property='EV'
fig, ax = plt.subplots(figsize=((3.5,1.8)))
fontsize=7
filt_df_monkey_median = filt_df_monkey.groupby(['Direction', 'Dataset Type', 'Date','Neuron distribution ID','control_shuffle'])['EV'].mean().reset_index()
fig4_violinplot(filt_df_monkey_median, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, label_order=directions, 
                hue_order=dataset_types, gap=-0.1,width=0.7,cut=0,
                linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
                )
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV Fraction', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax)
add_group_pairwise_stars_2_sets(filt_df_monkey, neuron_property, x=variable1, x_order=directions,
                       hue=variable2, hue_order=dataset_types, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
                       stars1_positions=[0.16,0.23,0.30],
                       stars2_positions=[0.69,0.77,0.83])
data = filt_df_monkey_null['EV']
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.1, color='blue', label='shuffle\ncontrol IQR',
        linewidth=0,
           )
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)
if save_figs is True:
    plt.savefig(fig_dir + f'fig4_monkey_{monkey_name}_dataset_comparisons_EV_control.pdf',transparent=True,bbox_inches='tight')
plt.show()



### supplemental

In [ ]:
variable1='Area'
variable2='Dataset Type'
dataset_types = ['RF_thin', 'RF_large', 'SNR']
directions= ['V4','V1']
figsize=(1.7,1.3)


In [ ]:
#Fig Supp 5L
stats_args = {'fontsize': fontsize*0.8, 'height1':1.02, 'height2': 1.12, 'height3': 1.02, 'stars1_positions': [0.18,0.26,0.34],'stars2_positions':[0.68,0.75,0.83],'bar_height':0.02 }
neuron_property='Split-half r'

fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_monkey, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=[0.8,0.9,1], ylabel='split-half r', animal='monkey', stats_args=stats_args)
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
        ax.set_ylim(top=1)
ax.plot([-0.2,0.2], # -0.095, units are width of the axes
        [1.101, 1.101], # 0 to 500, units are same as the y-axis
        clip_on=False,
        transform=ax.get_xaxis_transform(),
        color='black', linewidth=0.7)
ax.plot([0.804,1.2], # -0.095, units are width of the axes
        [1.101, 1.101], # 0 to 500, units are same as the y-axis
        clip_on=False,
        transform=ax.get_xaxis_transform(),
        color='black', linewidth=0.7)
if save_figs is True:
        plt.savefig(fig_dir + f'fig4_supp_monkey_{monkey_name}_dataset_comparisons_reli.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig Supp 5M
stats_args = {'fontsize': fontsize*0.8, 'height1':0.92, 'height2': 1.02, 'height3': 0.92, 
              'stars1_positions': [0.18,0.26,0.34],'stars2_positions':[0.68,0.75,0.83],'bar_height':0.02 }
neuron_property='SNR'
fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_monkey, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=None, ylabel=None, animal='monkey', stats_args=stats_args,
                                        log_scale=True)
# ax.set_ylim(top=100)
if save_figs is True:
    plt.savefig(fig_dir + f'fig4_supp_monkey_{monkey_name}_dataset_comparisons_SNR.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
#Fig Supp 5N
stats_args = {'fontsize': fontsize*0.8, 'height1':0.92, 'height2': 1.02, 'height3': 0.92,'stars1_positions': [0.18,0.26,0.34],'stars2_positions':[0.68,0.75,0.83],'bar_height':0.02 }
neuron_property='max corr. val'

fig, ax = plt.subplots(figsize=figsize)
plot_neuronal_properties_comparison(filt_df_monkey, variable1, neuron_property, variable2, 
                                        ax, fontsize=7, label_order=directions, hue_order=dataset_types, 
                                        yticks=None, ylabel='max corr. val.', animal='monkey', stats_args=stats_args)
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)
if save_figs is True:
    plt.savefig(fig_dir + f'fig4_supp_monkey_{monkey_name}_dataset_comparisons_max_corr_vals.pdf',transparent=True,bbox_inches='tight')
plt.show()

In [ ]:
# residuals
monkey_stats = all_monkey_stats['A']
df_monkey_all = make_monkey_df_stimulus_comparisons(monkey_stats, get_variances=True)
filt_df_monkey = df_monkey_all[(df_monkey_all.control_shuffle==False)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
filt_df_monkey_null= df_monkey_all[(df_monkey_all.control_shuffle==True)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)

dataset_types=['RF_thin', 'RF_large', 'SNR']
variable1='Direction'
variable2='Dataset Type'
directions = ['V1→V4','V4→V1']

# covs = ['SNR', 'Split-half r', ]
# 1) Control for many numeric covariates
covs = ['SNR', 'Split-half r',
       'Variance w/in trial\nacross timepoints',
       'Variance w/in timepoint\nacross trials']
# model on raw covariates
res_raw  = residualize_within_condition_multi(filt_df_monkey, y='EV', xs=covs, cond='Direction',
                                              flag_col='control_shuffle', flag_keep=False,
                                              standardize=True, model='ols')

In [ ]:
#Fig Supp 5O
neuron_property = 'EV_resid'
figsize=(1.7,1.3)

fig, ax = plt.subplots(figsize=figsize)
fontsize=7
fig4_violinplot(res_raw, x=variable1, y=neuron_property, 
                hue=variable2, ax=ax, label_order=directions, 
                hue_order=dataset_types, gap=-0.1,width=0.7,cut=0,
                linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
                )
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV residuals', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax)
add_group_pairwise_stars_2_sets(res_raw, neuron_property, x=variable1, x_order=directions,
                       hue=variable2, hue_order=dataset_types, ax=ax, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
                       stars1_positions=[0.16,0.23,0.30],
                       stars2_positions=[0.69,0.77,0.83])
# Get the y-axis ticks
y_ticks = plt.gca().get_yticks()
# Check if 1 is among the ticks
if 1 in y_ticks:
    ax.set_ylim(top=1)
ax.set_ylim(top=0.65)
if save_figs:
	plt.savefig(os.path.join(fig_dir, f'R2.6_macaque_A_ev_resid_covs_direction.pdf'), dpi=500, bbox_inches='tight', transparent=True)

## Monkey $L_{A}$ subsampled

In [ ]:

dataset_types=['RF_thin', 'RF_large', 'SNR']
variable1='Direction'
variable2='Dataset Type'
directions = ['V1→V4','V4→V1']
figsize=(3,2)
neuron_property = 'EV'


In [ ]:
from utils.stats_functions import get_group_pairwise_stars
def add_group_pairwise_stars_2_sets(df_, neuron_property, x, x_order, hue, hue_order, ax, fontsize=7, height1=0.95, height2=1.02,height3=0.95, 
stars1_positions=None, stars2_positions=None, bar_height=0.02, perm_t=True, perm_type='ind',central_tendency='median', hierarchical=False, return_p_values=False,
stars_dict = None, sort_keys=None, plot_stars = True):
    print(len(df_[(df_[x]==x_order[0])&(df_[hue]==hue_order[0])]), len(df_[(df_[x]==x_order[0])&(df_[hue]==hue_order[1])]))
    if stars_dict is None:
        star1_labels,all_stars1, pvals1 = get_group_pairwise_stars(df_[df_[x]==x_order[0]], hue, hue_order, neuron_property, perm_t=perm_t, perm_type=perm_type,
                                                                central_tendency=central_tendency, hierarchical=hierarchical, return_pvals=True, sort_keys=sort_keys)
        star2_labels, all_stars2, pvals2 = get_group_pairwise_stars(df_[df_[x]==x_order[1]], hue, hue_order, neuron_property, perm_t=perm_t, perm_type=perm_type,
                                                                central_tendency=central_tendency, hierarchical=hierarchical, return_pvals=True, sort_keys=sort_keys)
    else:
        all_stars1 = stars_dict[x_order[0]]['stars']
        all_stars2 = stars_dict[x_order[1]]['stars']
    if plot_stars is True:    
        height_positions = [height1,height2, height3]
        if stars1_positions is None:
            stars1_positions= [0.17,0.25,0.33]
        if stars2_positions is None:
            stars2_positions= [0.67,0.76,0.86]
        bar_halflength = 0.12

        for s, (star1, star2) in enumerate(zip(all_stars1, all_stars2)):
            if star1 =='n.s.':
                height1 = height_positions[s] + 0.018
                fontsize1 = fontsize*0.9
                color1='gray'
            else:
                height1 = height_positions[s]
                fontsize1=fontsize
                color1='black'
            if star2 =='n.s.':
                height2 = height_positions[s] + 0.018
                fontsize2 = fontsize*0.9
                color2='gray'
            else:
                height2 = height_positions[s]
                fontsize2 = fontsize
                color2='black'

            ax.text(stars1_positions[s], height1, star1, ha='center', va='center', fontsize=fontsize1, transform=ax.transAxes, color=color1)
            ax.text(stars2_positions[s], height2, star2, ha='center', va='center', fontsize=fontsize2, transform=ax.transAxes, color=color2)
            if s==1:
                if star1 is not None:
                    ax.hlines(height_positions[s]- bar_height, stars1_positions[s] - bar_halflength, stars1_positions[s] + bar_halflength, 
                            color=color1,transform=ax.transAxes, linewidth=0.5)
                if star2 is not None:
                    ax.hlines(height_positions[s]- bar_height, stars2_positions[s] - bar_halflength, stars2_positions[s] + bar_halflength, 
                            color=color2,transform=ax.transAxes, linewidth=0.5)
    if return_p_values is True:
        return {f'{x_order[0]}_pvals': pvals1, f'{x_order[1]}_pvals':pvals2, f'{x_order[0]}_labels':star1_labels, f'{x_order[1]}_labels':star2_labels}
                

In [ ]:

with open(os.path.join(project_root, 'results/fig_4', 'subsample_seeds.json'), 'r') as f:
    subsample_seeds = json.load(f)
    
main_monkey_name = 'L'
subsample_monkey_name = 'A'


stimulus_df_monkey_L_A = pd.DataFrame()
stimulus_df_monkey_L_A_null = pd.DataFrame()
pval_dict_all_seeds = {}
fontsize=7
for seed in subsample_seeds:
	subsampled_monkey_stats_path = os.path.join(project_root,f'results/fig_4/monkey_L_subsampled_to_{subsample_monkey_name}',f'monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_seed{seed}_stats.pkl')
	with open(subsampled_monkey_stats_path, 'rb') as f:
		subsampled_monkey_stats = pickle.load(f)
	all_datataset_types_ = [k for k in subsampled_monkey_stats.keys() if k not in ['meta','monkey_alphas','monkey_alphas_glm','monkey_directionality_alphas']]
	
	df_monkey_all = make_monkey_df_stimulus_comparisons(subsampled_monkey_stats)
	
	filt_df_monkey = df_monkey_all[(df_monkey_all.control_shuffle==False)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
	filt_df_monkey_null= df_monkey_all[(df_monkey_all.control_shuffle==True)&(df_monkey_all['Split-half r']>0.8)&(df_monkey_all.SNR>=2)].reset_index(drop=True)
	filt_df_monkey_mean = filt_df_monkey.groupby(['Direction', 'Dataset Type', 'Date','Neuron distribution ID','control_shuffle'])['EV'].mean().reset_index()
	filt_df_monkey_median_mean = filt_df_monkey_null.groupby(['Direction', 'Dataset Type', 'Date','Neuron distribution ID','control_shuffle'])['EV'].mean().reset_index()
	
	filt_df_monkey_mean['Subsample Seed'] = seed
	filt_df_monkey_median_mean['Subsample Seed'] = seed

	stimulus_df_monkey_L_A = pd.concat([stimulus_df_monkey_L_A, filt_df_monkey_mean], ignore_index=True)
	stimulus_df_monkey_L_A_null = pd.concat([stimulus_df_monkey_L_A_null, filt_df_monkey_median_mean], ignore_index=True)
	pvals_dict = add_group_pairwise_stars_2_sets(filt_df_monkey, neuron_property, x=variable1, x_order=directions,
						hue=variable2, hue_order=dataset_types, ax=None, fontsize=fontsize,height1=0.85, height2=0.95, height3=0.85,
						stars1_positions=[0.16,0.23,0.30],
						stars2_positions=[0.69,0.77,0.83], return_p_values=True, plot_stars=False)
	pval_dict_all_seeds[seed] = pvals_dict

In [ ]:
def p_to_stars(p):
	return "***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "n.s."))
pval_dict_median_across_seeds = {}
for direction in directions:
	if direction not in pval_dict_median_across_seeds:
		pval_dict_median_across_seeds[direction] = {}
		pval_dict_median_across_seeds[direction]['pvals'] = []
		pval_dict_median_across_seeds[direction]['labels'] = ['RF_thin_RF_large', 'RF_thin_SNR', 'RF_large_SNR']
	pvals_all_seeds = []
	for seed, pval_dicts in pval_dict_all_seeds.items():
		pvals_all_seeds.append(pval_dicts[f'{direction}_pvals'])
	pval_dict_median_across_seeds[direction]['pvals'] = np.median(np.array(pvals_all_seeds), axis=0)
	pval_dict_median_across_seeds[direction]['stars'] = [p_to_stars(p) for p in pval_dict_median_across_seeds[direction]['pvals']]


In [ ]:
# Fig Supp 5G
fig, ax = plt.subplots(figsize=((3.5,1.8)))
fig4_violinplot(stimulus_df_monkey_L_A, x=variable1, y=neuron_property, 
			hue=variable2, ax=ax, label_order=directions, 
			hue_order=dataset_types, gap=-0.1,width=0.7,cut=0,
			linewidth=0, saturation=1, palette=['lightseagreen','lightcoral','#72BEB7']
			)
ax.legend_.remove()
sns.despine()
ax.set_ylabel('EV Fraction', fontsize=fontsize, labelpad=0)
ax.tick_params(axis='both', labelsize=fontsize, width=0.3, length=2, pad=1)
sns.despine()
ax.spines[:].set_linewidth(0.3)
add_violin_custom_colors_3(ax)


data = stimulus_df_monkey_L_A_null['EV']
per_25 = np.percentile(data.dropna().values, 25)
per_75 = np.percentile(data.dropna().values, 75)
ax.axhspan(per_25, per_75, alpha=0.1, color='blue', label='shuffle\ncontrol IQR',
		linewidth=0,
		)

def p_to_stars(p):
	return "***" if p < 1e-3 else ("**" if p < 1e-2 else ("*" if p < 0.05 else "n.s."))
add_group_pairwise_stars_2_sets(filt_df_monkey_mean, neuron_property, x=variable1, x_order=directions,
						hue=variable2, hue_order=dataset_types, ax=ax, fontsize=fontsize,height1=0.9, height2=1, height3=0.9,
						stars1_positions=[0.16,0.23,0.30],
						stars2_positions=[0.69,0.77,0.83], stars_dict=pval_dict_median_across_seeds)
ax.set_yticklabels([])
ax.set_ylabel('')
if save_figs is True:
	plt.savefig(fig_dir + f'fig4_monkey_{main_monkey_name}_subsampled_to_{subsample_monkey_name}_dataset_comparisons_EV_control_all_seeds.pdf',transparent=True,bbox_inches='tight')
